<a href="https://colab.research.google.com/github/eddycc66/DEM-Fix-Interpolation-and-Contour-Generation/blob/master/Correccion_dem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install rasterio numpy scipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 100.3 MB/s eta 0:00:00


In [3]:
!unzip -q dem.zip -d dem

In [4]:
# Verificar que el DEM fue extraído correctamente
import glob
dem_files = glob.glob("dem/*.tif")
dem_path = dem_files[0]
print(f"Archivo DEM encontrado: {dem_path}")

Archivo DEM encontrado: dem/dem.tif


In [24]:
# Autor M.Sc. Edwin Calle Condori
import rasterio
import numpy as np
from scipy import ndimage
import glob

# Ruta automática al DEM extraído
dem_path = glob.glob("dem/*.tif")[0]
output_path = "dem_corregido_interp.tif"

# Leer el DEM
with rasterio.open(dem_path) as src:
    profile = src.profile
    dem = src.read(1)

    # Mostrar valores extremos antes
    print("Valores originales:", np.min(dem), "a", np.max(dem))

    # Definir NoData y actualizar perfil
    nodata_value = -9999
    profile.update(nodata=nodata_value)

    # Crear máscara de valores erróneos (valores muy negativos)
    mask = dem < -1000
    dem[mask] = np.nan  # Convertir a NaN para interpolación

    # Interpolación: rellenar NaN con vecinos válidos
    dem_interp = dem.copy()
    nan_mask = np.isnan(dem)
    dem_interp[nan_mask] = ndimage.generic_filter(dem, np.nanmean, size=5)[nan_mask]

# Guardar el DEM corregido e interpolado
with rasterio.open(output_path, 'w', **profile) as dst:
    dst.write(dem_interp.astype(rasterio.float32), 1)

print("DEM interpolado guardado como:", output_path)

Valores originales: -11831.721 a 5115.9736


/usr/local/lib/python3.11/dist-packages/scipy/ndimage/_filters.py:1963: RuntimeWarning: Mean of empty slice
  _nd_image.generic_filter(input, function, footprint, output, mode,


DEM interpolado guardado como: dem_corregido_interp.tif
